In [1]:
from IPython.display import display, Markdown, clear_output
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import PydanticOutputParser
from typing import List
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document
from langchain.output_parsers import OutputFixingParser
from operator import itemgetter
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI  # example LLM
from langchain.output_parsers import StructuredOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnablePassthrough, chain, RunnableConfig
import pandas as pd
import re
import os
from langchain_openai import OpenAI
from dotenv import load_dotenv
load_dotenv()




True

In [2]:

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=1.2,
    streaming=True
)


In [3]:
response = llm.invoke("Hello, world!")
print(response.content)


Hello! How can I assist you today?


In [4]:
#llm = OpenAI(name="o3")

In [5]:
#llm.invoke("Hello, world!")

In [7]:
prompt = PromptTemplate.from_template("""
    How to say {input} in {target_language} language
""")           

chain = prompt | llm | StrOutputParser()


chain.invoke({"input": "Danke", "target_language": "Russian"})

'To say "Danke" (which means "thank you" in German) in Russian, you would say "Спасибо" (pronounced "spasibo").'

In [8]:
folder_path = "documents"
files = os.listdir(folder_path)
print(files)

['Video', '.DS_Store', '1_1_Artikel.docx']


In [10]:
for filename in os.listdir(folder_path):
    print(os.path.join(folder_path, filename))

documents/Video
documents/.DS_Store
documents/1_1_Artikel.docx


In [15]:
from typing import List
import os
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_core.documents import Document

def load_documents(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    return documents

folder_path = "documents"
documents = load_documents(folder_path)

len(documents)

Unsupported file type: Video
Unsupported file type: .DS_Store


1

In [16]:
documents

[Document(metadata={'source': 'documents/1_1_Artikel.docx'}, page_content='DER ARTIKEL\n\nАртикль — это служебное слово, которое ставится перед существительным и показывает его род, число и падеж. В немецком языке артикли употребляются практически всегда. Различают определённый артикль, неопределённый артикль и отрицательный артикль.\n\nОпределённый артикль: der (мужской род), die (женский род), das (средний род), die (множественное число). Используется, когда речь идёт о чём-то конкретном, известном, уже упомянутом, уникальном или понятном из контекста. Примеры: der Mann, die Frau, das Kind, die Bücher.\n\nНеопределённый артикль: ein (мужской и средний род), eine (женский род). Используется при первом упоминании, если предмет неизвестен собеседнику. Не имеет формы во множественном числе. Примеры: ein Mann, eine Frau, ein Kind. Множественное: einfach Bücher (без артикля), einige Bücher, viele Bücher и т. д.\n\nОтрицательный артикль: kein, keine, kein. Склоняется как ein. Примеры: kein 

In [17]:
[doc.metadata for doc in documents]

[{'source': 'documents/1_1_Artikel.docx'}]

In [18]:
os.listdir("documents")

['Video', '.DS_Store', '1_1_Artikel.docx']

In [19]:
# Splitting documents into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=200)
chunks = splitter.split_documents(documents)
len(chunks)

6

In [20]:
print(chunks[5])

page_content='Вопрос: Когда можно сказать „die Deutschen“ без существительного? Ответ: Когда существительное заменяется субстантивированным прилагательным. „Die Deutschen“ = „die deutschen Menschen“.

Вопрос: Почему „das Neue“, но „ein neuer Tag“? Ответ: „Das Neue“ — субстантивированное прилагательное, ставшее существительным. Оно требует определённого артикля. „Ein neuer Tag“ — обычное сочетание с прилагательным.

Вопрос: Почему говорят „das Frühstück“, но „zu Frühstück“ — без артикля? Ответ: В устойчивом выражении „zu Frühstück“ артикль опущен по традиции, но правильно: „zum Frühstück“ (сокращение от „zu dem Frühstück“).

Вопрос: Почему говорят „Ich fahre mit dem Bus“, но „Ich gehe zu Fuß“? Ответ: „Bus“ — исчисляемый предмет, требует артикля. „Fuß“ в выражении „zu Fuß“ используется без артикля как часть устойчивого оборота.

Вопрос: Почему „ein paar Leute“ — с артиклем ein? Ответ: Здесь „ein paar“ — это устойчивое количественное выражение и переводится как „несколько“. Оно не связано

In [22]:
# chunk_40 = chunks[40]
print(chunks[5].page_content)


Вопрос: Когда можно сказать „die Deutschen“ без существительного? Ответ: Когда существительное заменяется субстантивированным прилагательным. „Die Deutschen“ = „die deutschen Menschen“.

Вопрос: Почему „das Neue“, но „ein neuer Tag“? Ответ: „Das Neue“ — субстантивированное прилагательное, ставшее существительным. Оно требует определённого артикля. „Ein neuer Tag“ — обычное сочетание с прилагательным.

Вопрос: Почему говорят „das Frühstück“, но „zu Frühstück“ — без артикля? Ответ: В устойчивом выражении „zu Frühstück“ артикль опущен по традиции, но правильно: „zum Frühstück“ (сокращение от „zu dem Frühstück“).

Вопрос: Почему говорят „Ich fahre mit dem Bus“, но „Ich gehe zu Fuß“? Ответ: „Bus“ — исчисляемый предмет, требует артикля. „Fuß“ в выражении „zu Fuß“ используется без артикля как часть устойчивого оборота.

Вопрос: Почему „ein paar Leute“ — с артиклем ein? Ответ: Здесь „ein paar“ — это устойчивое количественное выражение и переводится как „несколько“. Оно не связано с неопределён

In [16]:
# auto_label_prompt = PromptTemplate.from_template("""
# Come up with a short, general topic name for the following text.

# Text:
# {chunk}

# Return only the topic name.
# """)
# auto_label_chain = auto_label_prompt | llm | StrOutputParser()

# results = []

# for chunk in chunks[:10]:  
#     topic = auto_label_chain.invoke({
#         "chunk": chunk.page_content
#     })
#     results.append({
#         "text": chunk.page_content[:50],
#         "topic": topic
#     })

# import pandas as pd
# df = pd.DataFrame(results)
# df.tail()


In [23]:
# Creating embeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
model_kwargs = {'trust_remote_code': True
                # ,'device': 'cuda'
                }
#model_name='models/Linq-Embed-Mistral'
#model_name='models/multilingual-e5-large-instruct'
model_name='intfloat/multilingual-e5-large-instruct'
embedding_model = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
document_embeddings = embedding_model.embed_documents([split.page_content for split in chunks])
print(f"Created embeddings for {len(document_embeddings)} document chunks.")


README.md: 0.00B [00:00, ?B/s]

Created embeddings for 6 document chunks.


In [25]:
len(document_embeddings[0])

1024

In [26]:
# Creating vector store
from langchain_chroma import Chroma

# Setup collection
collection_name = "my_collection"
persist_directory = "./chroma_db"

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory=persist_directory,
    collection_name=collection_name
)
print(f"Vector store created and persisted to '{persist_directory}' (collection: '{collection_name}')")


Vector store created and persisted to './chroma_db' (collection: 'my_collection')


In [27]:
# Get retriever from the vector store
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# Formulate the query
query = "Когда употребляется неопределённый артикль?"

# Retrieve similar text chunks
search_results = retriever.invoke(query)

# Print the retrieved chunks
for i, result in enumerate(search_results, 1):
    print(f"Result {i}:")
    print(f"Source: {result.metadata.get('source', 'Unknown')}")
    print(f"Content: {result.page_content}")



Result 1:
Source: documents/1_1_Artikel.docx
Content: Вопрос: Почему в одном случае „Kaffee trinken“, а в другом „der Kaffee ist heiß“? Ответ: Без артикля — если речь идёт о действии в общем. С артиклем — при конкретизации или когда существительное является подлежащим.

Вопрос: Почему „Musik ist wichtig“, но „die Musik von Bach ist schön“? Ответ: Абстрактное понятие без артикля — когда говорим в общем. Определённый артикль — при уточнении, например, о чьей музыке идёт речь.

Вопрос: Когда используется артикль с названиями языков? Ответ: Без артикля — при указании, на каком языке говорят: „Ich spreche Spanisch“. С артиклем — если язык используется как подлежащее или описывается: „Das Spanische ist melodisch“.

Вопрос: Почему нельзя сказать „sie spricht das Deutsch“? Ответ: Потому что „Deutsch“ в этом контексте — имя языка, и оно употребляется без артикля. Артикль добавляется только при субстантивации: „das Deutsche“.

Вопрос: Почему нельзя сказать „Ich habe kein das Auto“? Ответ: Потому

In [30]:
#
from graph_retriever.strategies import Eager
from langchain_graph_retriever import GraphRetriever
traversal_retriever = GraphRetriever(
    store = vectorstore,
    edges = [
    ("артикли", "неопределенные"),     
    ("артилки", "определенные"),          
    ("существительные", "артикли"),           
],
    strategy = Eager(k=5, start_k=1, max_depth=2),
)

In [31]:
results = traversal_retriever.invoke("Зачем ставить артикль в немецком языке?")
for doc in results:
    print(f"{doc.id}: {doc.page_content}")

4bc55e1f-4c0a-4427-ac65-64eeac09fcc6: Вопрос: Почему в одном случае „Kaffee trinken“, а в другом „der Kaffee ist heiß“? Ответ: Без артикля — если речь идёт о действии в общем. С артиклем — при конкретизации или когда существительное является подлежащим.

Вопрос: Почему „Musik ist wichtig“, но „die Musik von Bach ist schön“? Ответ: Абстрактное понятие без артикля — когда говорим в общем. Определённый артикль — при уточнении, например, о чьей музыке идёт речь.

Вопрос: Когда используется артикль с названиями языков? Ответ: Без артикля — при указании, на каком языке говорят: „Ich spreche Spanisch“. С артиклем — если язык используется как подлежащее или описывается: „Das Spanische ist melodisch“.

Вопрос: Почему нельзя сказать „sie spricht das Deutsch“? Ответ: Потому что „Deutsch“ в этом контексте — имя языка, и оно употребляется без артикля. Артикль добавляется только при субстантивации: „das Deutsche“.

Вопрос: Почему нельзя сказать „Ich habe kein das Auto“? Ответ: Потому что в немецком 

In [39]:
prompt = ChatPromptTemplate.from_template(
    """Answer the question ONLY based on the context below.'

Context:
{context}

Question:
{question}


Не выводи никакие специальные символы, текст должен быть легко читаемым

"""
)

In [32]:
def format_docs(docs):
    return "\n\n".join(f"text: {doc.page_content} metadata: {doc.metadata}" for doc in docs)
parser = StrOutputParser()
llm_chain = (
        {"context": traversal_retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | parser
    )

# print(llm_chain.invoke(f'Какого рода слова употребляются без артикля?'))

In [33]:
output = ""

async for event in llm_chain.astream_events(""):
    if event["event"] == "on_chat_model_stream":
        chunk = event["data"]["chunk"]
        output += chunk.content
        clear_output(wait=True)
        display(Markdown(output))


KeyError: "Input to PromptTemplate is missing variables {'target_language', 'input'}.  Expected: ['input', 'target_language'] Received: ['question', 'context']\nNote: if you intended {target_language} to be part of the string and not a variable, please escape it with double curly braces like: '{{target_language}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "

In [36]:
from IPython.display import display, Markdown, clear_output

output = ""

async for event in llm.astream_events("Расскажи шутку про Германию"):
    if event["event"] == "on_chat_model_stream":
        chunk = event["data"]["chunk"]
        output += chunk.content  # теперь правильно
        clear_output(wait=True)
        display(Markdown(output))





Конечно! Вот шутка про Германию:

Почему в Германии нет комиков из числа автомобилистов?

Потому что в Германии каждая дорожная ситуация строго по правилам и управлению, ну очень трудно "влезть" с неожиданным поворотом сюжета!

In [37]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),  # ← твоя база
    return_source_documents=True
)

result = qa_chain.invoke({"query": "Когда используется неопределённый артикль?"})
print(result["result"])


Неопределённый артикль используется в нескольких случаях:

1. **При первом упоминании предмета или лица.** Когда вы говорите о чём-то неизвестном или в первый раз: "Ich habe ein Buch gelesen." (Я прочитал книгу.)

2. **Для обозначения чего-то неопределённого, одного из многих.** Например: "Ich suche eine Wohnung." (Я ищу квартиру.)

3. **С профессиями и национальностями.** Например: "Er ist ein Lehrer." (Он учитель.)

4. **Когда речь идёт о представителе категории.** Например: "Ein Hund braucht Bewegung." (Любой/всякий/один из собак нуждается в движении.)

Это также помогает отличить что-то общее от конкретного, когда определённого артикля (der/die/das) ещё не уместно.


In [ ]:
# async for event in llm_chain.astream_events("Зачем ставить артикль в немецком языке?"):
#     if event["event"] == "on_chat_model_stream":
#         print(event["data"]["chunk"], end="", flush=True)


KeyError: "Input to PromptTemplate is missing variables {'target_language', 'input'}.  Expected: ['input', 'target_language'] Received: ['question', 'context']\nNote: if you intended {target_language} to be part of the string and not a variable, please escape it with double curly braces like: '{{target_language}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "

In [ ]:
# async for event in llm_chain.astream_events("Зачем ставить артикль в немецком языке?"):
#     kind = event["event"]
#     #print(kind)
#     if kind == "on_chat_model_stream":
#         print(event, end="|", flush=True)

KeyError: "Input to PromptTemplate is missing variables {'target_language', 'input'}.  Expected: ['input', 'target_language'] Received: ['question', 'context']\nNote: if you intended {target_language} to be part of the string and not a variable, please escape it with double curly braces like: '{{target_language}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "

In [41]:
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
parser = StrOutputParser()
chain = prompt | llm | parser

async for event in chain.astream_events({"topic": "parrot"}):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(event, end="|", flush=True)

{'event': 'on_chat_model_stream', 'data': {'chunk': AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run--15687ad6-e7a3-4788-810a-dc9ebdb050f5')}, 'run_id': '15687ad6-e7a3-4788-810a-dc9ebdb050f5', 'name': 'ChatOpenAI', 'tags': ['seq:step:2'], 'metadata': {'ls_provider': 'openai', 'ls_model_name': 'gpt-4o', 'ls_model_type': 'chat', 'ls_temperature': 1.2}, 'parent_ids': ['d1c33537-ddb2-469a-9bf2-c4ffa2fdb761']}|{'event': 'on_chat_model_stream', 'data': {'chunk': AIMessageChunk(content='Why', additional_kwargs={}, response_metadata={}, id='run--15687ad6-e7a3-4788-810a-dc9ebdb050f5')}, 'run_id': '15687ad6-e7a3-4788-810a-dc9ebdb050f5', 'name': 'ChatOpenAI', 'tags': ['seq:step:2'], 'metadata': {'ls_provider': 'openai', 'ls_model_name': 'gpt-4o', 'ls_model_type': 'chat', 'ls_temperature': 1.2}, 'parent_ids': ['d1c33537-ddb2-469a-9bf2-c4ffa2fdb761']}|{'event': 'on_chat_model_stream', 'data': {'chunk': AIMessageChunk(content=' do', additional_kwargs={}, response_metada

In [42]:
def output_structure(x):
    return parser.get_format_instructions()

In [44]:
class Answer(BaseModel):
    answer: str = Field(description="Answer to the question")
    references: List[str] = Field(description="List of references to the answer")

parser = PydanticOutputParser(pydantic_object=Answer)
parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

prompt = ChatPromptTemplate.from_template(
    """Answer the question based only on the context provided.

    Context: {context}

    Question: {question}

Structure of an answer: {instructions}

"""
    )

llm_chain = (
        {"context": traversal_retriever | format_docs, "question": RunnablePassthrough(), "instructions": output_structure}
        | prompt
        | llm
        | parser
    )



In [45]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"answer": {"description": "Answer to the question", "title": "Answer", "type": "string"}, "references": {"description": "List of references to the answer", "items": {"type": "string"}, "title": "References", "type": "array"}}, "required": ["answer", "references"]}\n```'

In [46]:
df = pd.DataFrame(llm_chain.invoke("Как класть тротуарную плитку?")).T
df.columns = df.iloc[0]
df

,answer,references
0,answer,references
1,Контекст не содержит информацию по данной теме.,[]


In [35]:
df.to_csv("answer.csv")

In [36]:
df.iloc[0]

0
answer            answer
references    references
Name: 0, dtype: object